In [ ]:
from typing import List, Union
import pandas as pd
import numpy as np
import ast
import optuna
import csv



from rdkit import Chem

import torch
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATv2Conv



from sklearn.model_selection import train_test_split

from tqdm import tqdm


from joblib import Parallel, delayed


In [ ]:
class FeaturizationParameters:
    def __init__(self):
        self.max_atomic_num = 100
        self.atom_features = {
            'atomic_num': list(range(self.max_atomic_num)),
            'degree': [0, 1, 2, 3, 4, 5],
            'formal_charge': [-1, -2, 1, 2, 0],
            'chiral_tag': [0, 1, 2, 3],
            'num_Hs': [0, 1, 2, 3, 4],
            'hybridization': [
                Chem.rdchem.HybridizationType.SP,
                Chem.rdchem.HybridizationType.SP2,
                Chem.rdchem.HybridizationType.SP3,
                Chem.rdchem.HybridizationType.SP3D,
                Chem.rdchem.HybridizationType.SP3D2
            ],
        }
        self.atom_fdim = sum(len(choices) + 1 for choices in self.atom_features.values()) + 2

def onek_encoding_unk(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

def atom_features(atom, params):
    features = onek_encoding_unk(atom.GetAtomicNum() - 1, params.atom_features['atomic_num']) + \
               onek_encoding_unk(atom.GetTotalDegree(), params.atom_features['degree']) + \
               onek_encoding_unk(atom.GetFormalCharge(), params.atom_features['formal_charge']) + \
               onek_encoding_unk(int(atom.GetChiralTag()), params.atom_features['chiral_tag']) + \
               onek_encoding_unk(int(atom.GetTotalNumHs()), params.atom_features['num_Hs']) + \
               onek_encoding_unk(int(atom.GetHybridization()), params.atom_features['hybridization']) + \
               [1 if atom.GetIsAromatic() else 0] + \
               [atom.GetMass() * 0.01]  # scaled to about the same range as other features
    return features

PARAMS = {
    'BOND_FDIM': 10
}

def bond_features(bond: Chem.rdchem.Bond) -> List[Union[bool, int, float]]:
    if bond is None:
        fbond = [1] + [0] * (PARAMS['BOND_FDIM'] - 1)
    else:
        bt = bond.GetBondType()
        fbond = [
            0,  # bond is not None
            bt == Chem.rdchem.BondType.SINGLE,
            bt == Chem.rdchem.BondType.DOUBLE,
            bt == Chem.rdchem.BondType.TRIPLE,
            bt == Chem.rdchem.BondType.AROMATIC,
            bond.GetIsConjugated() if bt is not None else 0,
            bond.IsInRing() if bt is not None else 0
        ]
        fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))
    return fbond


class MoleculeData:
    def __init__(self, smiles, target, addHs=True):
        self.smiles = smiles
        self.target = torch.tensor(target, dtype=torch.float)
        self.mol = Chem.MolFromSmiles(smiles)
        if addHs:
            self.mol = Chem.AddHs(self.mol)
        self.params = FeaturizationParameters()
        self.edge_index, self.edge_attr = self.construct_graph()

    def construct_graph(self):
        edge_index = []
        edge_attr = []
        for bond in self.mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            edge_index.extend([[start, end], [end, start]])
            edge_attr.extend([bond_features(bond), bond_features(bond)])  # Добавляем признаки для обеих направлений связи
        return torch.tensor(edge_index).t().contiguous(), torch.tensor(edge_attr, dtype=torch.float)

    def generate_atom_features(self):
        features = []
        for atom in self.mol.GetAtoms():
            features.append(atom_features(atom, self.params))
        return torch.tensor(features, dtype=torch.float)

class MoleculeDataset(Dataset):
    def __init__(self, dataframe, smiles_column='smiles', target_column='target', addHs=True, n_jobs=-1):
        super(MoleculeDataset, self).__init__()
        self.data_list = Parallel(n_jobs=n_jobs)(
            delayed(lambda row: MoleculeData(row[smiles_column], row[target_column], addHs))(
                row) for _, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]))

    def len(self): 
        return len(self.data_list)

    def get(self, idx):
        molecule_data = self.data_list[idx]
        x = molecule_data.generate_atom_features()
        edge_index = molecule_data.edge_index
        edge_attr = molecule_data.edge_attr
        y = molecule_data.target
        
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
        data.smiles = molecule_data.smiles
        
        return data


In [ ]:
molecule_dataset = torch.load('../data/QM_137k.pt')
print(molecule_dataset)
for i in range(2):
    print(molecule_dataset[i])

In [ ]:
indices = list(range(len(molecule_dataset)))
train_val_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.125, random_state=42)  # 0.125 * 0.8 = 0.1

train_dataset = [molecule_dataset[i] for i in tqdm(train_indices, desc="Creating Train Dataset")]
val_dataset = [molecule_dataset[i] for i in tqdm(val_indices, desc="Creating Validation Dataset")]
test_dataset = [molecule_dataset[i] for i in tqdm(test_indices, desc="Creating Test Dataset")]

batch_size = 128

dataset = DataLoader(molecule_dataset, batch_size=batch_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)


in_features = 133                # Количество входных признаков для каждого узла
out_features =  1                # Количество выходных признаков
num_epochs = 1000                # Количество эпох обучения
learning_rate = 0.005            # Скорость обучения
weight_decay = 5e-4              # Вес распада для регуляризации
warmup_epochs = 2                
initial_lr = learning_rate  
patience = 10
best_val_loss = float('inf')
epochs_no_improve = 0

hidden_features = 64             # Количество скрытых признаков
num_heads = 1                    # Количество "голов" в механизме внимания
dropout_rate = 0.1                 # Процент дропаута

# Определение модели
class GATv2Model(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_heads, dropout_rate):
        super(GATv2Model, self).__init__()
        self.conv1 = GATv2Conv(in_channels=in_features, out_channels=hidden_features,
                               heads=num_heads, dropout=dropout_rate, concat=True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_features * num_heads)
        self.conv2 = GATv2Conv(in_channels=hidden_features * num_heads, out_channels=out_features, 
                               heads=1, concat=False, dropout=dropout_rate)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(self.bn1(x))
        x = F.dropout(x, training=self.training, p=dropout_rate)
        x = self.conv2(x, edge_index)
        return x.squeeze()

model = GATv2Model(
        in_features=in_features,
        hidden_features=hidden_features,
        out_features=out_features,
        num_heads=num_heads,
        dropout_rate=dropout_rate
    )

model.to(device)
print(f"Model:\n{model}")

# Определение функции потерь и оптимизатора
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

print('Start Train')



In [ ]:
def save_trial_to_csv(trial_number, trial_value, trial_params, csv_path='optuna_results.csv'):
    with open(csv_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            headers = ['Trial', 'Value'] + [key for key in trial_params.keys()]
            writer.writerow(headers)
        row = [trial_number, trial_value] + list(trial_params.values())
        writer.writerow(row)

def train_model(trial):
    # Гиперпараметры, подлежащие оптимизации
    hidden_features = trial.suggest_int('hidden_features', 32, 1024, step=32)
    num_heads = trial.suggest_int('num_heads', 1, 9)
    dropout_rate = trial.suggest_float('dropout_rate', 0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    # еще есть торч лайтнинг

    # Создание и обучение модели
    model = GATv2Model(in_features, hidden_features, out_features, num_heads, dropout_rate).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        if epoch < warmup_epochs:
            lr = learning_rate * (epoch + 1) / warmup_epochs
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        else:
            lr = learning_rate

        model.train()
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = loss_func(out, batch.y)
            loss.backward()
            optimizer.step()

        # Валидация
        model.eval()
        val_loss_accum = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validating', leave=False):
                batch.to(device)
                val_out = model(batch.x, batch.edge_index)
                val_loss = loss_func(val_out, batch.y)
                val_loss_accum += val_loss.item()

        avg_val_loss = val_loss_accum / len(val_loader)
        
        # Ранняя остановка
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                break

    return best_val_loss

def objective(trial):
    best_val_loss = train_model(trial)
    save_trial_to_csv(trial.number, best_val_loss, trial.params)
    return best_val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print(f' Value: {trial.value}')
print(' Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')